<a href="https://colab.research.google.com/github/Fil8/Apertif_modules/blob/master/Can_you_make_notes_with_bullet_points_and_command_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Summary of the key steps and logic implemented to generate the TNG50-like HDF5 file with original and rotated coordinates/velocities, focusing on the `fromFlashToTNG.py` script:

---
## Summary: Generating TNG50-like HDF5 from FLASH Data

This outlines the process to convert FLASH simulation data, apply filters, calculate halo orientation, and save data in a structure resembling TNG50 snapshots, including rotated kinematic fields.

### 1.  Script Setup and Argument Parsing ⚙️
* **Import necessary libraries:** `yt`, `h5py`, `numpy`, `argparse`, `os`.
* **Command-line arguments:**
    * `-i, --input`: Path to the input FLASH HDF5 file. (e.g., `cca/bondi_hdf5_plt_cnt_0083.h5`)
    * `-o, --output`: Path for the output TNG50-like HDF5 file.
    * `-tmin, --temperature_min`: Minimum temperature filter (Kelvin).
    * `-tmax, --temperature_max`: Maximum temperature filter (Kelvin).
    * `--sphere_radius`: Optional spherical filter radius (in code length units).
    * `--cosmo_header_file`: Optional path to an HDF5 file for overriding cosmological parameters in the output header.

---
### 2.  Loading FLASH Data and Initial Setup 💾
* Load the FLASH dataset using `yt.load(args.input)`.
* Print basic dataset information (time, redshift, domain dimensions, code unit conversions to CGS).
* Define primary FLASH field names (e.g., `('flash', 'dens')`, `('flash', 'velx')`) and `yt`-derived field names (e.g., `('index', 'cell_volume')`).
* **Field Checking:**
    * Verify the existence of essential FLASH fields.
    * Attempt to use common `yt` aliases (e.g., `('gas', 'density')` if `('flash', 'dens')` is missing) and update field variables accordingly.
* **Temperature Field:** Define/ensure a consistent temperature field (e.g., `('gas', 'temperature')` via the `_temperature` helper function) for filtering, ensuring it uses `('flash', 'temp')` if available.

---
### 3.  Data Selection and Filtering 🔍
* Start with `selection = ds.all_data()`.
* Apply temperature cuts if `args.temperature_min` or `args.temperature_max` are provided:

In [ ]:
# Example for tmin
    selection = selection.cut_region(f"obj[('gas', 'temperature')] >= {temp_min_for_filter.value}")

* Apply an optional spherical filter if `args.sphere_radius` is provided:

In [ ]:
sphere_region_filt = ds.sphere(ds.domain_center, radius_yt_quantity)
    selection = selection.intersect(sphere_region_filt)

* Check if `num_selected_cells > 0` before proceeding to save.

---
### 4.  Calculating Orientation and Rotation (Core of New Logic) 🌀
*This occurs if `do_save_output` is `True`.*
* **Calculate CoM and Bulk Velocity of the `selection`:**

In [ ]:
com_selection = selection.quantities.center_of_mass(use_gas=True).in_units(_length_unit)
    bulk_velocity_selection = selection.quantities.bulk_velocity(use_gas=True).in_units(_velocity_unit)

* **Calculate Principal Axes Rotation Matrix (`principal_axes_R`):**
    * This is done by the helper function `calculate_principal_axes_rotation_matrix(ds, selection, com_selection)`.
    * Inside this function:
        * Fetch `('gas', 'cell_mass')` and absolute positions `('gas', 'x')`, `('gas', 'y')`, `('gas', 'z')` for the `selection`.
        * Calculate positions relative to `com_selection`.
        * Manually compute the components of the **inertia tensor** ($I_{xx}, I_{yy}, I_{zz}, I_{xy}, I_{xz}, I_{yz}$) using these relative positions and masses.
        * Perform an eigen decomposition of the unitless inertia tensor matrix: `eigenvalues_val, eigenvectors_matrix = np.linalg.eig(inertia_tensor_val)`.
        * Sort eigenvectors by descending eigenvalues. The resulting matrix `principal_axes_R` has these sorted principal axes as its **columns**.
        * Return `principal_axes_R` and a flag indicating if a non-identity rotation was successfully calculated.

---
### 5.  Preparing and Transforming Kinematic Data 💨
* **Fetch Original (Box-Frame) Data:**
    * `coordinates_code_val`: Nx3 NumPy array of selected cell positions (values in code units).
    * `velocities_code_val`: Nx3 NumPy array of selected cell velocities (values in code units).
* **Transform Coordinates:**
    1.  Make coordinates relative to the `selection`'s CoM:

In [ ]:
coords_relative_to_com_val = coordinates_code_val - com_selection.value

2.  Rotate these relative coordinates. If `principal_axes_R` has the new axes (P1, P2, P3) as columns, then for row vectors `v_orig`, the transformed vector `v_rot = v_orig @ principal_axes_R`.

In [ ]:
rotated_coordinates_val = np.dot(coords_relative_to_com_val, principal_axes_R)

The `rotated_coordinates_val` are now in the principal axis frame, centered at the CoM of the selection.
* **Transform Velocities:**
    1.  Make velocities relative to the `selection`'s bulk velocity:

In [ ]:
vels_relative_to_bulk_val = velocities_code_val - bulk_velocity_selection.value

2.  Rotate these relative velocities using the same `principal_axes_R`:

In [ ]:
rotated_velocities_val = np.dot(vels_relative_to_bulk_val, principal_axes_R)

* If `rotation_calculated` is `False` (e.g., identity matrix from helper due to error/no cells), `rotated_coordinates_val` and `rotated_velocities_val` are set to the CoM/bulk-velocity-relative versions without further matrix multiplication.

---
### 6.  Preparing Other Physical Quantities 🌡️
* Fetch and convert to code units (as `YTArray`s or values):
    * `density_code_ytarray`
    * `masses_code_ytarray` (from density \* cell volume)
    * `temperature_K_ytarray`
    * `internal_energy_code_ytarray` (optional, with error handling)
    * `particle_ids` (generated via `np.arange`)

---
### 7.  Saving Data to TNG50-like HDF5 File  HDF5 📝
* Open the output HDF5 file using `h5py.File(output_filename, 'w')`.
* **Header Group (`/Header`):**
    * Read cosmological parameters (`HubbleParam`, `Omega0`, etc.) from `args.cosmo_header_file` if provided.
    * Fall back to `ds` (if cosmological) or TNG defaults for any missing parameters.
    * Store these `final_cosmo_params` as attributes.
    * Store `BoxSize`, `UnitLength_in_cm`, `UnitMass_in_g`, `UnitVelocity_in_cm_per_s`.
    * Store other TNG header fields like `MassTable`, `NumPart_ThisFile`, flags (e.g., `Flag_DoublePrecision` set to 1 if coordinates are `float64`).
* **Particle Data Group (`/PartType0`):**
    * `Coordinates`: Original box-frame positions (`coordinates_code_val`), typically saved as `np.float64`.
    * `Velocities`: Original box-frame velocities (`velocities_code_val`), typically saved as `np.float32`.
    * `RotatedCoordinates`: Transformed coordinates (`rotated_coordinates_val`), same precision as `Coordinates`.
    * `RotatedVelocities`: Transformed velocities (`rotated_velocities_val`), same precision as `Velocities`.
    * `Masses`, `Density`, `Temperature`, `InternalEnergy` (if available), `ParticleIDs`.
* **Cosmology Text File:** Save the `final_cosmo_params` to a separate `_cosmology.txt` file for easy reference.

---
### 8.  Error Handling ⚠️
* `try...except` blocks are used for file operations, `yt` calls, and calculations to catch errors gracefully (e.g., `YTFieldNotFound`, `LinAlgError` during eigen decomposition).

This detailed breakdown covers the journey from loading raw FLASH data to producing a TNG50-like HDF5 file with both original and properly rotated kinematic fields based on the halo's principal axes of shape.